# ClingARC Pipelines

In [1]:
from utils import load_prompts, wholesale_solve, test_primitives, iterative_solve, build_and_eval

import os
from llm import LLM

api_key = os.getenv("OPENAI_API_KEY")
llm = LLM(api_key)

instance = "train_instances/1"

### Wholesale pipeline

1. read (asp_program + facts)
2. 'primitives' = run_clingo + print_models
3. 'rule_explanation' = wholesale_pass('primitives')
4. read (asp_program + test_facts)
5. 'test_input_primitives' = run_clingo + print_models
6. test_pass('rule_explanation','test_input_primitives')

In [2]:
primitives_text = wholesale_solve(instance) #finds primitives in examples (except test input)
print(primitives_text)

Results for grid3.lp:
Input grid:
 grid_size(input,3,3) horizontal_line(input,start(0,1),end(0,2),color(gray),length(2)) point(input,(1,0),gray) point(input,(2,1),gray) vertical_line(input,start(0,2),end(1,2),color(gray),length(2))

Output grid:
 grid_size(output,3,3) horizontal_line(output,start(0,0),end(0,2),color(yellow),length(3)) horizontal_line(output,start(1,0),end(1,2),color(green),length(3)) horizontal_line(output,start(2,0),end(2,2),color(yellow),length(3))


Results for grid2.lp:
Input grid:
 grid_size(input,3,3) horizontal_line(input,start(1,0),end(1,1),color(gray),length(2)) horizontal_line(input,start(2,0),end(2,1),color(gray),length(2)) point(input,(0,2),gray) vertical_line(input,start(0,0),end(2,0),color(gray),length(3)) vertical_line(input,start(1,1),end(2,1),color(gray),length(2))

Output grid:
 grid_size(output,3,3) horizontal_line(output,start(0,0),end(0,2),color(red),length(3)) horizontal_line(output,start(1,0),end(1,2),color(yellow),length(3)) horizontal_line(outp

In [ ]:
primitives_text = wholesale_solve(instance) #finds primitives in examples (except test input)
# print(primitives_text)
print('-- Primitives found in examples')

rule_explanation = llm.call("wholesale_pass", primitives=primitives_text, track_usage=False) #LLM call to predict transformation rules in examples

print(f'Transformation rules predicted for examples:{rule_explanation}\n')

test_instance_path = os.path.join(instance, "grid_test.lp") 

test_input_primitives = test_primitives(test_instance_path) #finds primitives in test input
print('-- Primitives found in test input --')

# final LLM call to predict test output from test input primitives and predicted rules
test_pass = llm.call("test_pass", rule_explanation=rule_explanation, test_input_primitives=test_input_primitives, track_usage=False)
print(f'Test output predicted:{test_pass}\n')

### Iterative pipeline

1. read (asp_program + facts)
2. 'primitives_list' = run_clingo + print_models
3. for set in 'primitives_list':
    'rule_explanation' = iterative_pass(set)
    keep refining rule_explanation
4. read (asp_program + test_facts)
5. 'test_input_primitives' = run_clingo + print_models
6. test_pass('rule_explanation','test_input_primitives')

In [14]:
instance = 'train_instances/1'

In [15]:
primitives_list = iterative_solve(instance) #finds primitives in examples (except test input)
primitives_list

['Results for grid3.lp:\nInput grid:\n grid_size(input,3,3) point(input,(0,1),gray) point(input,(1,2),gray) point(input,(2,1),gray)\n\nOutput grid:\n grid_size(output,3,3) horizontal_line(output,start(0,0),end(0,2),color(yellow),length(3)) horizontal_line(output,start(1,0),end(1,2),color(green),length(3)) horizontal_line(output,start(2,0),end(2,2),color(yellow),length(3))\n\n========================================\n',
 'Results for grid2.lp:\nInput grid:\n grid_size(input,3,3) point(input,(0,0),gray) point(input,(1,1),gray) point(input,(2,0),gray)\n\nOutput grid:\n grid_size(output,3,3) horizontal_line(output,start(0,0),end(0,2),color(red),length(3)) horizontal_line(output,start(1,0),end(1,2),color(yellow),length(3)) horizontal_line(output,start(2,0),end(2,2),color(red),length(3))\n\n========================================\n',
 'Results for grid1.lp:\nInput grid:\n grid_size(input,3,3) vertical_line(input,start(0,2),end(2,2),color(gray),length(3))\n\nOutput grid:\n grid_size(output,3

In [21]:
prompts = load_prompts("prompts.yaml")
print(prompts['iterative_pass']['user'])

Previous input-output grid primitives:
{previous_primitives}

Previous predicted rule explanation:
{previous_rule_explanation}

Input-output pair primitives {idx} out of {n_examples}:
{primitives}



In [ ]:
for idx, model in enumerate(primitives_list):

    print(f'--- Iteration {idx+1} of {len(primitives_list)} ---')
    n_examples = len(primitives_list)
    seen_primitives = str(primitives_list[:idx])
    previous_rule_explanation = rule_explanation if idx > 0 else "N/A"

    if idx == 0:
        rule_explanation = llm.call("first_pass", n_examples=n_examples, primitives=model, track_usage=False) #LLM call to predict transformation rules in examples

    else:
        rule_explanation = llm.call("iterative_pass", 
                                    previous_primitives=seen_primitives, 
                                    previous_rule_explanation=previous_rule_explanation, 
                                    idx=idx,
                                    n_examples=n_examples,
                                    primitives=model,
                                    track_usage=False) #LLM call to predict transformation rules in examples


    print(f'Transformation rules predicted for example:{rule_explanation}\n')


--- Iteration 1 of 4 ---
--- Iteration 2 of 4 ---
--- Iteration 3 of 4 ---
--- Iteration 4 of 4 ---


In [24]:
print(rule_explanation)

**Re-evaluation of Previous Predictions Based on New Input-Output Pair:**

1. **Input Grid Description:**
   - The grid is 3x3.
   - There are three gray points located at (0,2), (1,1), and (2,0).

2. **Output Grid Description:**
   - The grid remains 3x3.
   - Three horizontal lines fill each row:
     - Row 0 is filled with green.
     - Row 1 is filled with yellow.
     - Row 2 is filled with red.

**Analysis of New Example Against Previous Patterns:**

- The new input-output pair further confirms that when isolated gray points are present in the input grid, each row in the output grid is filled with horizontal lines of alternating colors (similar to previous instances with points).
- The presence of isolated points results in varied colors for each row rather than a uniform color as seen with vertical input lines.

**Refined Rule Description:**

1. **Grid Size:**
   - The grid size in both input and output remains 3x3.

2. **Transformation Process:**
   - Each row is consistently t

In [26]:
test_input_primitives = test_primitives('train_instances/1/grid_test.lp') #finds primitives in test input


test_pass = llm.call("test_pass", rule_explanation=rule_explanation, test_input_primitives=test_input_primitives, track_usage=False)


In [27]:
print(test_pass)

Based on the refined rule description and the given input grid for the test case, we need to apply the identified transformation process to generate the output grid. Let's break down the steps:

### Test Input Grid:

- **Grid Size**: 3x3
- **Points**: 
  - (0,2), color: gray
  - (1,0), color: gray
  - (2,1), color: gray

### Transformation Process:

1. **Grid Size**: The output grid maintains the same size as the input grid, which is 3x3.

2. **Output Grid Structure**: Each row of the input grid will correspond to a fully filled horizontal line in the output grid. 

3. **Color Assignment**:
   - Row 0 will be filled with the first color in the sequence: Green.
   - Row 1 will be filled with the second color in the sequence: Yellow.
   - Row 2 will be filled with the third color in the sequence: Red.

### Applying the Transformation:

- **Row 0**: Will be fully filled with Green.
- **Row 1**: Will be fully filled with Yellow.
- **Row 2**: Will be fully filled with Red.

### Constructing

### Translation & Evaluation

In [6]:
from utils import is_executable_asp

In [3]:
test_pass = """
Assumption (from the provided rule set): counts map to colours as 1 → green, 2 → yellow, 3 → red. No full-height vertical input line is present, so no override applies.

Input row counts:
- Row 0: 1 gray cell
- Row 1: 1 gray cell
- Row 2: 1 gray cell

Apply rule: each output row becomes a full horizontal stripe whose colour is the colour for that row's count (count=1 → green). Therefore every row is green and the entire 3×3 output is filled green.

Output primitives:
- grid_size(output,3,3)
- horizontal_line(output,(0,0),(0,2),green,3))
- horizontal_line(output,(1,0),(1,2),green,3)
- horizontal_line(output,(2,0),(2,2),green,3)

Visual (rows shown top→bottom): 
[green, green, green]
[green, green, green]
[green, green, green]
"""


In [4]:
test_input_primitives = test_primitives('train_instances/1/grid_test.lp') #finds primitives in test input
print(test_input_primitives)

Input grid:
 grid_size(input,3,3) point(input,(0,2),gray) point(input,(1,0),gray) point(input,(2,1),gray)

Output grid:
 




In [10]:

asp_output_grid = llm.call("translate_to_asp", test_input_primitives=test_input_primitives,
                           test_output_prediction=test_pass, 
                           track_usage=False) #LLM call to predict transformation rules in examples


In [16]:
print(asp_output_grid)

grid_size(output,3,3).
horizontal_line(output,start(0,0),end(0,2),color(green)).
horizontal_line(output,start(1,0),end(1,2),color(green)).
horizontal_line(output,start(2,0),end(2,2),color(green)).


In [7]:
test_syntax_error = """grid_size(output,3,3).
horizontal_line(output,start(0,0),end(0,2),color(green)).
horizontal_line(output,start(1,0),end(1,2),color(green)).
horizontal_line(output,start(2,0),end(2,2),color(green))."""

#### is_executable logic block, fixes syntax errors

In [8]:
outcome = is_executable_asp(test_syntax_error)
if outcome == True:
    print("The ASP code is executable and produces output.")
else:
    print("Not executable:")
    revised_asp = llm.call("fix_asp_code", error_message=outcome, 
                           test_input_primitives=test_input_primitives,
                            test_output_prediction=test_pass,
                           track_usage=False)

    print(f'Revised ASP code:\n{revised_asp}')
    if is_executable_asp(revised_asp) == True:
        print("The revised ASP code is now executable and produces output.")
    else:
        print("The revised ASP code is still not executable.")

The ASP code is executable and produces output.


In [9]:
print(asp_output_grid)

NameError: name 'asp_output_grid' is not defined

In [ ]:
i = """
- grid_size(output,3,3)
- horizontal_line(output,(0,0),(0,2),green,3))
- horizontal_line(output,(1,0),(1,2),green,3)
- horizontal_line(output,(2,0),(2,2),green,3)
"""

if is_executable_asp(i):
    print("The ASP code is executable and produces output.")

else:
    revised_asp = llm.call("fix_asp_code", error_message=i, track_usage=False)

### build grid

In [1]:
from utils import build_and_eval

In [2]:
executable_output = """
grid_size(output,3,3).
horizontal_line(output,start(0,0),end(0,2),color(green)).
horizontal_line(output,start(1,0),end(1,2),color(red)).
horizontal_line(output,start(2,0),end(2,2),color(yellow)).
point(output,(2,0),color(green)).
"""

In [3]:
result = build_and_eval(prediction=executable_output)

In [4]:
result = build_and_eval(prediction=executable_output)

if result == 'correct_grid':
    print("The predicted output grid matches the actual output grid. Success!")
else:
    print("The predicted output grid does not match the actual output grid. Failure.")
    print(f"Here are the attempted primitives:\n{executable_output}")
    print(f"Here are the errors in the output:\n{result}")

The predicted output grid does not match the actual output grid. Failure.
Here are the attempted primitives:

grid_size(output,3,3).
horizontal_line(output,start(0,0),end(0,2),color(green)).
horizontal_line(output,start(1,0),end(1,2),color(red)).
horizontal_line(output,start(2,0),end(2,2),color(yellow)).
point(output,(2,0),color(green)).

Here are the errors in the output:
incorrect_cell((2,0),(predicted(green),correct(yellow))) incorrect_cell((2,0),duplicate(green,yellow)) incorrect_cell((2,0),duplicate(yellow,green)) incorrect_grid
